In [4]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [6]:
import torch
import pandas as pd
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

#enc_output = 32 x 60 x 768 형태의 tensor 데이터
#label = 32 x 1 x 1 형태의 tensor 데이터 

In [ ]:
#768차원의 cls 토큰을 6개 라벨로 분류한 후
#그 중 제일 큰 값을 가지는 라벨로 매핑
#ex) cls 토큰(768차원) --- 감성 분류 후 --> label1(1차원) ====> 768차원으로 늘려주기 
class NNModel(nn.Module):
    def __init__(self, input_dim, class_size):
        super(NNModel,self).__init__()
        self.l1 = nn.Linear(input_dim,520) #768차원의 cls token을..
        self.l2 = nn.Linear(520,320)
        self.l3 = nn.Linear(320,240)
        self.l4 = nn.Linear(240,120)
        self.l5 = nn.Linear(120, class_size)   #6개로 classification하기 위해..
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [ ]:
#######################변수 선언#########################

batch_size = 32
# total_batch_num = len(총 챗봇 데이터 개수) / batch_size
train_batch_size = 4 #32개를 train하기 위해 임의로 설정
input_dim = 768
class_size = 6
epochs = 10000
lr = 0.01

#######################변수 선언#########################


device = "cuda" if torch.cuda.is_available() else "cpu"
model = NNModel(input_dim, class_size).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)

for big_batch_num in total_batch_num:
  # enc_outputs(tensor) 선언 - encoder에서 output을 받아..
  # labels(tensor) 선언 - 해당 시퀀스의 label을 받아..
  train_dataset = TensorDataset(enc_outputs, labels)
  train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

  for epoch in range(epochs+1):
    cost = 0.0
    for (step, batch) in enumerate(train_dataloader):
      model.train()
      batch = tuple(t.cuda() for t in batch)
      optimizer.zero_grad()

      enc_output, label = batch
      enc_output = enc_output.view(-1, input_dim).to(device)
      label = label.to(device)

      hypothesis = model(enc_output)
      loss = criterion(hypothesis, label)
      loss.backward()
      optimizer.step()

      cost += loss
  if (epoch+1)%100==0:
      print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

In [ ]:
#######################변수 선언#########################

sentiment_labels = {0: "label0", 1: "label1", 2: "label2", 3: "label3", 4: "label4", 5: "label5"}
# test_data(tensor) 선언 - enc_outputs 중 랜덤으로 가져와..
# labels(tensor) 선언 - 해당 test 데이터의 label을 가져와..
test_batch_size = 4

#######################변수 선언#########################

with torch.no_grad():
    model.eval()
    
    test_dataset = TensorDataset(test_data, labels)
    test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=test_batch_size)

    for (step, batch) in enumerate(test_dataloader):
      batch = tuple(t.cuda() for t in batch)
      enc_output, label = batch

      enc_output = enc_output.view(-1, input_dim).to(device)
      label = label.to(device)

      prediction = model(enc_output)

      print('--------------------------------')
      print(prediction)
      print(torch.round(F.softmax(prediction, dim=1), decimals=2))
      print(prediction.argmax(1))
      print(list(map(sentiment_labels.get, prediction.argmax(1).tolist())))